### IMPORT
- 자산정리 월별 차이점 정리
  - 4월 : 확실한 종료(종결, 매각, 환매, 차주면책+정상보증인없음)만 종료로 처리
  - 5월 : 할당제외한 모든 종료,불가사유 종료로 처리(제외, 포기, 시효, [차주 : 사망,이민,말소,파산,법인]+정상보증인유무에 따라 보증인있음 or 종료처리요망)
  - 시효 정리되면 그것만 제대로 반영하면 됨

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
credit_dtype = {'채무자키':str, '계좌키':str, '보증인키':str, '심의차수':str, '변제금\n수취계좌':str}
event_dtype = {'채무자키':str, '계좌키':str, '법조치키':str, '관할법원코드':str, '타채무자키':str,'타법조치키':str}

In [2]:
wd = r"D:\3.자산\전산 dataset"
basedate = "230531"

comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"종결|매각|환매|DS|완납|완제|종료", 
    # 시효
    "시효" : r"시효|부활",
    # 채무자별로 체크
    "면책" : r"면책|면탈",  
    "불가" : r"제외|포기|파산(?<!\(기각\))|사망|이민|말소"
    }

### 파일 읽기

In [21]:
# # 개별 파일 pkl 만들기
# ext = ".xlsx"
# credit = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="확정,미확정", dtype=credit_dtype).fillna("")
# # 타입 등 후처리
# credit.replace({pd.NaT: ""}, inplace=True)
# credit["상환후잔액"] = credit["상환후잔액"].replace("",0).astype(np.int64)
# # 저장 : 파일명 수정
# credit.to_pickle(join(wd, basedate, files[4] +".pkl"))

In [4]:
# 파일 읽기
# 채무자, 계좌, 보증인, 개회신복 기준데이터->차주와 보증인으로 나누 저장

################### 예수금 지울것(채무자,계좌)
files = ("채무자조회새창_20230531_1837", "계좌조회새창_20230531_1841", "보증인새창_20230531_1851_새보증인상태", "개인회생새창_20230531_1846_진행중_계좌키분리_기준데이터", "신용회복전체리스트_20230531_솔림")
# 개회의 겨우 계좌키 분리 파일로 읽을 것! 

# 파일 읽기
ext = ".pkl"  
###################

# 개회-신복 충돌 정리(기준데이터 만들 때 각 담당자가 제거하기로 함)
#### 개회제외 = {"차주" : ("20462830"), "보증인" : () } # 20462830 중복입금건. 신복이 기간이 더 남아서 신복으로 처리하기로 함
#### 신복제외 = {"차주" : ("20451534", "20525880", "20525532"), "보증인" : ()} # "20451534", "20525880", "20525532" 신복 실효 확정 전에 개회 신청한 건
if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    debt = pd.read_pickle(join(wd, basedate, files[0]+ext))
    account = pd.read_pickle(join(wd, basedate, files[1]+ext))
    grt = pd.read_pickle(join(wd, basedate, files[2]+ext))
    rehabilitation = pd.read_pickle(join(wd, basedate, files[3]+ext))
    credit = pd.read_pickle(join(wd, basedate, files[4]+ext))
else : 
    debt = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=debt_dtype).fillna("")
    account = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=account_dtype).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=grt_dtype).fillna("")
    rehabilitation = pd.read_excel(join(wd, basedate, files[3]+ext), sheet_name="개인회생(진행)", dtype=rehabilitation_dtype).fillna("")
    credit = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="확정,미확정", dtype=credit_dtype).fillna("")

# 엑셀저장시 NaT 를 ""로
credit.replace({pd.NaT: ""}, inplace=True)
# 신용회복 상환후잔액 타입 int로 
credit["상환후잔액"] = credit["상환후잔액"].replace("",0).astype(np.int64)
# 조정채권 분리 (충돌 확인 위해 필요함)
# copy() 하면 pd.NaT도 또 해주지 않아도 된다.
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"].copy()
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"].copy()
credit_d = credit[credit["채무구분"]=="주채무"].copy()
credit_grt = credit[credit["채무구분"]=="보증채무"].copy()
# 예수금 제외
debt.drop(debt[debt.성명=="예수금"].index, inplace=True)
debt.reset_index(drop=True, inplace=True)
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account.reset_index(drop=True, inplace=True)

In [13]:
# # pkl 저장
debt.to_pickle(join(wd, basedate, files[0]+".pkl"))
account.to_pickle(join(wd, basedate, files[1]+".pkl"))
grt.to_pickle(join(wd, basedate, files[2]+".pkl"))
rehabilitation.to_pickle(join(wd, basedate, files[3]+".pkl"))
credit.to_pickle(join(wd, basedate, files[4]+".pkl"))

### 파일 수정
- 파산사건결과 -> 채무상태 반영 : 사건결과 없으면 보증인이나 승계인을 대상으로 한 것일 가능성이 높으므로 차주상태 변경하면 안 돼
- 시효(계좌비고에 "채권시효만료") : 계산된 시효는 남은 경우 있고, 대부분 종료건. 파일 수정 없이 시효 체크시에만 사용

In [5]:
origin_debt = debt[["채무자키", "채무상태"]].copy()
pattern = re.compile('[가-힣]+')

for i, v in debt[["채무자키", "채무상태", "파산사건결과"]].iterrows() : # 담당자는 필요 없다. 어차피 채무상태가 종료건 아닌 것만 바꿀 것이므로
    if v.파산사건결과 != "" and re.search(comp_closing["종료"], v.채무상태)==None :
        debt.loc[i, "채무상태"] = "파산("+ pattern.search(v.파산사건결과)[0]+ ")"

# 확인 ###########
merge = pd.merge(origin_debt, debt[["채무자키", "채무상태", "파산사건결과"]], how='outer', on='채무자키', suffixes=('_ori', '_new'))
# merge[merge.파산사건결과 != ""]
merge[merge.채무상태_ori != merge.채무상태_new]

,채무자키,채무상태_ori,채무상태_new,파산사건결과
1659,20429740,파산(진행중),파산(폐지),2022-08-24 폐지
1688,20466441,파산(면책),파산(폐지),2022-02-17 폐지
3358,20408754,정상,파산(인용),2018-06-27 인용
4704,20429846,파산(면책),파산(폐지),2022-05-19 폐지
5846,20427531,파산(면책),파산(인용),2022-04-25 인용
8961,20430128,기타,파산(인용),2020-03-04 인용
17052,20412185,파산(면책),파산(폐지),2022-08-22 폐지
17053,20442374,파산(면책),파산(폐지),2022-08-22 폐지
18103,20413808,파산(면책),파산(인용),2012-11-01 인용
21140,20420066,파산(면책),파산(폐지),2022-08-01 폐지


### 개회, 신복 충돌 체크 
- 종료, 비정상건과 충돌해도 삭제 x. 
  - 풀별 파일 만들때 새채무상태가 조정인 채무자키만 참조하면 되고, 매각후에도 계속 봐야하므로 기준데이터에 남긴다.
  - 비정상으로 분류했지만, 부활이거나 상태값이 잘못된 건의 경우 조정채권이 될 수도 있음. 환매 없앨려면 이건 따로 리스트 만들어 내부 정리하는 게 매각리스트로 뽑는 거 보다 낫겠다.

In [6]:
# [check]개회신복 채무자키 중복 : 차주, 보증인별

# 차주
조정중복_차주 = pd.merge(rehabilitation_d[["채무자키","계좌키","분납키","인가/미인가","반영OPB", "접수일", "분납자관계"]], credit_d[["채무자키","계좌키","접수번호","진행구분","반영OPB","접수일자","채무구분"]], on=["채무자키","계좌키"], how="inner")
display("차주", 조정중복_차주)

# 보증인
조정중복_보증인 = pd.merge(rehabilitation_grt[["채무자키","계좌키","분납자성명인","인가/미인가","반영OPB", "접수일"]].rename(columns={"분납자성명인":"고객명"}), credit_grt[["채무자키","계좌키","고객명","진행구분","반영OPB", "접수일자"]], on=["채무자키","계좌키", "고객명"], how="inner")
display("보증인", 조정중복_보증인)

'차주'

,채무자키,계좌키,분납키,인가/미인가,반영OPB_x,접수일,분납자관계,접수번호,진행구분,반영OPB_y,접수일자,채무구분
0,20451534,200974027,20141394,미인가,9213605,2023-03-06,채무자,20_077535,확정,13909130,2020-07-27,주채무
1,20502201,201029879,20144062,미인가,1432473,2023-05-16,채무자,21_101053,확정,1584537,2021-10-15,주채무
2,20502202,201029880,20144063,미인가,1872428,2023-05-16,채무자,21_101053,확정,2126622,2021-10-15,주채무
3,20525532,201059197,20141675,미인가,12467920,2023-03-28,채무자,22_005216,확정,12644899,2022-01-14,주채무
4,20525880,201059545,20141674,미인가,7643021,2023-03-28,채무자,22_005216,확정,7727495,2022-01-14,주채무


'보증인'

,채무자키,계좌키,고객명,인가/미인가,반영OPB_x,접수일,진행구분,반영OPB_y,접수일자


In [7]:
# 접수일자에 따라서 신복, 개회 하나만 남기기 
print(f'삭제전 개회 : {len(rehabilitation)}, 삭제전 신복 : {len(credit)}')
if len(조정중복_차주)>0 :
    개회index, 신복index = [], []
    for i, v in 조정중복_차주.iterrows() :
        if datetime.strptime(v.접수일, "%Y-%m-%d") > v.접수일자 : # 개회접수일 나중인 경우, 신복에서 제거
            신복index.append(credit[((credit.채무자키 == v.채무자키) & (credit.계좌키 == v.계좌키))].index.values[0])
        else : 개회index.append(rehabilitation[((rehabilitation.채무자키 == v.채무자키) & (rehabilitation.계좌키 == v.계좌키))].index.values[0])
    print(신복index)
    print(개회index)
    if len(신복index)>0 : credit = credit[~credit.index.isin(신복index)]
    if len(개회index)>0 : rehabilitation = rehabilitation[~rehabilitation.index.isin(개회index)]

if len(조정중복_보증인) > 0 :
    개회index, 신복index = [], []
    for i, v in 조정중복_보증인.iterrows() :
        if datetime.strptime(v.접수일, "%Y-%m-%d") > v.접수일자 : # 개회접수일 나중인 경우, 신복에서 제거
            신복index.append(credit[((credit.채무자키 == v.채무자키) & (credit.계좌키 == v.계좌키)) & credit.고객명 == v.고객명].index.values[0])
        else : 개회index.append(rehabilitation[((rehabilitation.채무자키 == v.채무자키) & (rehabilitation.계좌키 == v.계좌키)), rehabilitation.분납자성명인 == v.고객명].index.values[0])
    print(신복index)
    print(개회index)
    if len(신복index)>0 : credit = credit[~credit.index.isin(신복index)]
    if len(개회index)>0 : rehabilitation = rehabilitation[~rehabilitation.index.isin(개회index)]
print(f'삭제후 개회 : {len(rehabilitation)}, 삭제후 신복 : {len(credit)}')

삭제전 개회 : 5853, 삭제전 신복 : 15456
[3875, 5387, 5388, 15437, 15436]
[]
삭제후 개회 : 5853, 삭제후 신복 : 15451


In [8]:
# 확정된 조정채권 차주, 보증인 저장하기. 종결건에도 따로 문제발생해서 처리했다면 이 셀 재 실행 해야..
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"].copy()
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"].copy()
credit_d = credit[credit["채무구분"]=="주채무"].copy()
credit_grt = credit[credit["채무구분"]=="보증채무"].copy()

### grt, 새보증인상태 추가
- 새보증인상태 들어간 파일을 읽었다면 PASS

In [84]:
# 개회 >> 차주 : 계좌키, 보증인 : [계좌키 + 보증인이름]이 pk. 
# 신복 >> 보증인키 있으므로 이걸로. 누락계좌는 정상으로 나옴
# 개회 누락채권(일부계좌누락, 일부 채무자키(이런 경우가 있을까?)) 문제는 어떻게? : 일단 

# 보증인 상태변경
if "새보증인상태" in grt.columns : grt["새보증인상태"] = ""
else : grt.insert(8,"새보증인상태","")

# 종료, 면책, 종료권고 채우기 (보증인의 경우 폐업-> 종료권고)
cond1 = grt.종결일 != ""
cond2 = grt.보증인상태.str.contains(comp_closing["종료"])
cond3 = grt.보증인상태.str.contains(comp_closing["시효"]) # 보증인 상태값에도 시효가 있네..check
cond4 = grt.보증인상태.str.contains(comp_closing["면책"])
cond5 = grt.보증인상태.str.contains(comp_closing["불가"])
cond6 = grt.보증인상태.str.contains("폐업")
condlist_grt = [cond1, cond2, cond3, cond4, cond5, cond6, True]
valuelist_grt = ["종료", "종료", "시효", "면책", "불가", "불가", ""]
grt["새보증인상태"] = np.select(condlist_grt, valuelist_grt )

# '채무자키'열과 '보증인성명인'열을 기준으로 비교하여 값 변경
for i in tqdm(range(len(grt)), total=len(grt)):  
    # 새보증인상태 빈값인 경우(조정/정상)
    if grt.at[i, '새보증인상태'] == "" :
        # 개인회생
        temp = rehabilitation_grt[(rehabilitation_grt['채무자키'] == grt.at[i, '채무자키']) & (rehabilitation_grt['분납자성명인'] == grt.at[i, '보증인성명인'])]
        if len(temp) > 0:
            grt.at[i, '새보증인상태'] = "개인회생(확정)" if temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)"
        else : 
            # 신용회복
            temp = credit_grt[(credit_grt['보증인키'] == grt.at[i, '보증인키'])]
            if len(temp) > 0:
                grt.at[i, '새보증인상태'] = "신용회복("+ temp['상환방식'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
            else :
                # 무담보
                grt.at[i, '새보증인상태'] = '정상'

100%|██████████| 10312/10312 [00:03<00:00, 2723.00it/s]


In [85]:
# 저장 : 보증인 수정할 것 엑셀 저장
# 수정해야 함 : grt[grt.새보증인상태 != grt.보증인상태].loc[:,["채무자키", "계좌키","보증인키","매각사구분","보증인성명인","주민번호인","계좌번호","보증구분","새보증인상태","보증인상태","보증금액","보증잔액","보증한도액"]].to_excel(join(wd, basedate, "9.보증인상태변경할것.xlsx"), index=False)
# # 새보증인상태 추가한 파일 저장
grt.to_pickle(join(wd, basedate, files[2]+"_새보증인상태.pkl"))
grt.to_excel(join(wd, basedate, files[2]+"_새보증인상태.xlsx"), index=False)

### 전체 원데이터 만들기(종합)

In [9]:
raw_data = pd.DataFrame(None)
raw_data["채무자키"] = account.채무자키
raw_data["계좌키"] = account.계좌키
raw_data["타채무자키"] = account.타채무자키

raw_data["채권구분"] = account.채권구분 ##
raw_data["채권상태"] = "" ##
raw_data["새채무상태"] = "" ##
raw_data["채무상태"] = account.채무상태 ##
raw_data["담당자"] = account.담당자 ##
raw_data["성명"] = account.채무자명 #
raw_data["주민등록번호"] = account.채무자주민번호 # #.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["매각사구분"] = account.매각사구분
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["최초연체이자율"] = account.최초연체이자율
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계 ##
raw_data["최초대출일"] = account.최초대출일
raw_data["대출만기일"] = account.대출만기일
raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음 없으면 대출만기일로 대체해야.
raw_data["매입일"] = account.매입일
# 시효관련 추가
raw_data['시효완성월'] = np.where((account['시효중단여부'] == 'Y'), '중단', account['시효완성월'])
raw_data["시효사유"] = account.시효연장사유 # 시효연장사유 빈칸인 경우, 최초완성일메모(거의), 최초시효완성일, 시효중단여부 모두 빈칸임
raw_data["시효완성일"] = account.시효완성일
raw_data["종결일"] = account.종결일
raw_data["메모"] =account.메모
# 채무자조회에서 불러올 항목
debt_temp = debt[["채무자키", "개인법인", "연령", "자택우편번호", "자택주소", "관리자비고"]].rename(columns={"관리자비고":"새채무자키"})
raw_data = pd.merge(raw_data, debt_temp, on="채무자키", how="left")

# grt에서 각 계좌별 (새)보증인상태 불러오기
names = grt.groupby('계좌키')['보증인성명인'].apply(lambda x: ','.join(x)) # 고유값만 남기려면 join의 매개변수로 x.unique()를 사용
newstatus = grt.groupby('계좌키')['새보증인상태'].apply(lambda x: ','.join(x))
status = grt.groupby('계좌키')['보증인상태'].apply(lambda x: ','.join(x))

raw_data = raw_data.merge(names, how='left', on='계좌키')
raw_data = raw_data.merge(newstatus, how='left', on='계좌키')
raw_data = raw_data.merge(status, how='left', on='계좌키')

raw_data["보증인성명인"] = raw_data["보증인성명인"].fillna('')
raw_data["새보증인상태"] = raw_data["새보증인상태"].fillna('')
raw_data["보증인상태"] = raw_data["보증인상태"].fillna('')

#### 채권상태, 새채무상태 작성
- 채권상태 : 종료, 비정상(종료처리할것,보증인정상,시효), 정상
- 새채무상태 : 채권상태 정상이면 개인, 법인, 개인회생(진행중), 개인회생(확정), 신용회복(개인), 신용회복(프리), 신용회복(진행중)

In [10]:
채권상태, 새채무상태 = [], []
종료 = re.compile(comp_closing["종료"])
시효 = re.compile(comp_closing["시효"])
면책 = re.compile(comp_closing["면책"])
불가 = re.compile(comp_closing["불가"])

for i, v in tqdm(raw_data[["채무상태", "담당자", "종결일", "메모", "새보증인상태", "계좌키", "개인법인"]].iterrows(), total = len(raw_data)) :
    state = v.채무상태 + v.담당자
    # 종료
    if (v.종결일 != "") or 종료.search(state) : 
        새채무상태.append("종료")
        채권상태.append("종료")
    elif v.메모 != "" : # 현재 메모에는 '종료'만 있으나 "비정상"도 기입할 수 있으므로 별도 처리함.
        새채무상태.append(v.메모)
        if v.메모 == "종료" : 채권상태.append("종료")
        else : 채권상태.append("비정상")
    # 비정상 / 종료
    else :
        # 비정상1 - 시효
        if 시효.search(state):
            새채무상태.append("시효")
            채권상태.append("비정상")
        # 비정상2 - 차주 면책 또는 불가
        elif 면책.search(state) or 불가.search(state):
            # 종료,면책,불가,시효,',', " "를 제외하니 남는 게 없다면
            result = re.sub("종료|면책|불가|시효|,|\s", "", v.새보증인상태)
            if result == "" :
                새채무상태.append("종료처리요망")
            # 보증인정상
            else : 
                새채무상태.append("보증인정상")
            채권상태.append("비정상")
        # 정상 : 법인 별도 처리 
        else :
            temp채권상태 = "정상"
            # 차주 개인회생
            개회temp = rehabilitation_d[rehabilitation_d['계좌키'] == v.계좌키]
            if  len(개회temp) > 0:
                새채무상태.append("개인회생(확정)" if 개회temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)")
            else : 
                # 차주 신용회복
                신복temp = credit_d[credit_d['계좌키'] == v.계좌키]
                if len(신복temp) > 0:
                    새채무상태.append("신용회복("+ 신복temp['상환방식'].values[0]+")" if 신복temp['진행구분'].values[0] == "확정" else "신용회복(진행중)")
                # 무담보(개인,법인)
                else :
                    if v.개인법인 == "개인" : 
                        새채무상태.append(v.개인법인)
                    # 법인인 경우 보증인 상태 재확인
                    else :
                        result = re.sub("종료|면책|불가|시효|,|\s", "", v.새보증인상태)
                        # 정상 보증인 없음
                        if result == "" :
                            새채무상태.append("종료처리요망")
                            temp채권상태 = "비정상"
                        # 보증인정상
                        else : 
                            새채무상태.append("법인")
            채권상태.append(temp채권상태)
                        
# 작성
raw_data["채권상태"] = 채권상태
raw_data["새채무상태"] = 새채무상태

100%|██████████| 78157/78157 [00:34<00:00, 2256.47it/s] 


In [53]:
# [check] 종결처리 해야 하는 건 저장하기 - 코드 수정해야
# cond1 = raw_data.종결일 == ""
# cond2 = (raw_data.채무상태.str.contains(comp_closing["면책"]) | raw_data.담당자.str.contains(comp_closing["면책"])) & ~(raw_data.새보증인상태.str.contains("정상|개인회생|신용회복"))
# raw_data[cond1 & cond2].to_excel(join(wd, basedate, "[check]종결처리할것-완적면책.xlsx"))
# [ckeck]종결처리해야하는 계좌 출력
# if len(종결처리안된종료건)>0 :
#     # print(종결처리안된종료건)
#     pd.DataFrame(종결처리안된종료건).to_excel(join(wd, basedate, "[check]종결처리안된종료건.xlsx"), index=False)
# else : print("이상무")

In [65]:
# [ckeck]계좌별 채무상태 수정할 것 엑셀 저장
# 범주가 다르게 적용되어서 현재는 의미가 없다. 보완이 필요
# raw_data[raw_data.새채무상태 != raw_data.채무상태].loc[:,["채무자키", "계좌키","매각사구분","새채무상태","채무상태"]].to_excel(join(wd, basedate, "[check]계좌별채무상태달라진것.xlsx"), index=False)

#### 반영OPB 작성하기
- 비정상도 정상과 마찬가지로 무담보와 조정채권 OPB를 각각 반영한다.(내부용으로 봐야하니까)
- 종결은 모두 0으로 처리한다

In [11]:
raw_data = raw_data.drop(columns=raw_data.columns[raw_data.columns.str.contains("반영OPB")])

# 반영OPB 작성하기
# 1. 조정채권 OPB 입력
개회OPB = rehabilitation_d.loc[:, ["계좌키", "반영OPB"]]
신복OPB = credit_d.loc[:, ["계좌키", "반영OPB"]]
# 개회OPB와 신복OPB 병합 (반영OPB_X, _Y가 되지않도록 ON = [])
merged_opb = 개회OPB.merge(신복OPB, on=['계좌키','반영OPB'], how='outer')
# raw_data와 병합하며 "반영OPB"칼럼 만들기
raw_data = raw_data.merge(merged_opb, on='계좌키', how='outer')

# 2. 조정채권 아닌 것 OPB로 채우기
raw_data.loc[raw_data['반영OPB'].isnull(), '반영OPB'] = raw_data.loc[raw_data['반영OPB'].isnull(), "OPB"]

# 3. 종료건이면 0원으로 처리하기
raw_data["반영OPB"] = np.where(raw_data['채권상태'] == '종료', 0, raw_data['반영OPB'])

# 자료형 : 정수
raw_data["반영OPB"] = raw_data["반영OPB"].astype(np.int64)

In [12]:
# 컬럼 순서 
raw_data.insert(8, "새보증인상태", raw_data.pop('새보증인상태'))
raw_data.insert(9, "반영OPB", raw_data.pop('반영OPB'))

In [14]:
# 전체 저장하기
with pd.ExcelWriter(join(wd, basedate, "2.솔림헬프_전체_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    raw_data.to_excel(writer, sheet_name="리스트", index=False)
# 특정 매각사만 저장하기
# cond = "스마트저축-01" ###################################
# raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_원데이터_"+basedate+".xlsx"), index=False)

#### check. 채무상태 충돌

In [15]:
# [check] 동일차주 더블 채무상태(새채무자키, 주민번호 모두 보자)
새채무자키_채무자별채무상태 = raw_data.groupby('새채무자키')['새채무상태'].apply(lambda x:",".join(x.unique()))
check_새채무자키_채무상태 = 새채무자키_채무자별채무상태[새채무자키_채무자별채무상태.str.contains(',')]
주민번호_채무자별채무상태 = raw_data.groupby('주민등록번호')['새채무상태'].apply(lambda x:",".join(x.unique()))
check_주민번호_채무상태 = 주민번호_채무자별채무상태[주민번호_채무자별채무상태.str.contains(',')]
# 출력
if len(check_새채무자키_채무상태) > 0 : 
    check_새채무자키_채무상태.to_excel(join(wd, basedate, "[check]더블채무상태_새채무자키.xlsx"))
else : print("새채무자키 기준 이상무")
if len(check_주민번호_채무상태) > 0 : 
    check_주민번호_채무상태.to_excel(join(wd, basedate, "[check]더블채무상태_주민번호.xlsx"))
else : print("주민번호 기준 이상무")
print(len(check_새채무자키_채무상태), len(check_주민번호_채무상태))

356 580


In [16]:
# 종결건 제외하기(개회, 신복 따로 작업하기)
종결된계좌키 = raw_data[raw_data.채권상태 == "종료"].계좌키
# 종결건 확인
개회_종결 = rehabilitation[rehabilitation.계좌키.isin(종결된계좌키)][["채무자키", "계좌키"]].copy()
신복_종결 = credit[credit.계좌키.isin(종결된계좌키)][["채무자키", "계좌키"]].copy()
개회_종결["구분"] = "개회"
신복_종결["구분"] = "신복"
# 기본정보 추가
조정_종결 = pd.merge(개회_종결, 신복_종결, on=["채무자키", "계좌키", "구분"], how='outer')
조정_종결 = pd.merge(조정_종결, raw_data[["계좌키","채권상태","새채무상태","채무상태","담당자"]], on='계좌키', how='left')
# 매각 건 제외
print(f'매각건 제외 전 : {len(조정_종결)}')
매각idx = 조정_종결[(조정_종결.채무상태.str.contains('매각')) | (조정_종결.담당자.str.contains('매각'))].index
조정_종결 = 조정_종결.drop(매각idx)
print(f'매각건 제외 후 : {len(조정_종결)}')
display(조정_종결)
# [check]종결건 저장
# if len(조정_종결)>0 : 
#     조정_종결.to_excel(join(wd,basedate, "[check]매각외종결건vs조정.xlsx"), index=False)

매각건 제외 전 : 13
매각건 제외 후 : 6


,채무자키,계좌키,구분,채권상태,새채무상태,채무상태,담당자
2,20515090,201045943,신복,종료,종료,종결,신용회복
8,20463398,200983448,신복,종료,종료,종결,신용회복
9,20463216,200983266,신복,종료,종료,종결,신용회복
10,20461763,200981813,신복,종료,종료,종결,신용회복
11,20463258,200983308,신복,종료,종료,종결,신용회복
12,20465128,200985178,신복,종료,종료,종결,신용회복


### 신복, 개회 원데이터 만들기

In [17]:
# 제출용 시트 작성하기-개인회생 
re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
# re_d.insert(0, "순번", range(1,len(re_d)+1))
# re_grt.insert(0, "순번", range(1,len(re_grt)+1))
# 저장
with pd.ExcelWriter(join(wd, basedate, "2.솔림헬프_개인회생_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    re_d.to_excel(writer, sheet_name="차주", index=False)
    re_grt.to_excel(writer, sheet_name="보증인", index=False)

In [19]:
# 제출용 시트 작성하기-신용회복
# 기준데이터에서 삭제한 것이 있을 때 유의미 & 차주 보증인 구분

# # "NaT" 처리 - 제대로 됐으면 안 해도 되는건데.. 뭔가 꼬였을 때
# credit_d.replace({"NaT": ""}, inplace=True)
# credit_grt.replace({"NaT": ""}, inplace=True)

# 날짜열 문자열로
date_cols = credit_d.select_dtypes(include='datetime').columns
credit_d[date_cols] = credit_d[date_cols].astype(str)
date_cols = credit_grt.select_dtypes(include='datetime').columns
credit_grt[date_cols] = credit_grt[date_cols].astype(str)

# # 저장
with pd.ExcelWriter(join(wd, basedate, "2.솔림헬프_신용회복_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    credit_d.to_excel(writer, sheet_name="차주", index=False)
    credit_grt.to_excel(writer, sheet_name="보증인", index=False)

In [23]:
# 요약 
pd.options.display.float_format = '{:.0f}'.format
# 특정 POOL
# pool계좌키 = pd.read_excel("c://users/sl/Desktop/계좌키.xlsx", dtype={"계좌키":str})
# pool = pd.merge(raw_data,pool계좌키, on='계좌키', how='inner')
# result = pool.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])
# result
# 전체
# reindex에 있는 것만 집계되네 
raw_data.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','보증인정상','시효','종료처리요망','종료'])

,계좌키,반영OPB
새채무상태,,
개인,10264,114606013430
법인,1671,158735303164
개인회생(확정),5460,12899764567
개인회생(진행중),297,2289530290
신용회복(개인),8938,21480597571
신용회복(프리),5962,39310941266
신용회복(진행중),425,2804393458
보증인정상,613,13505262737
시효,331,21896203860


### POOL별 엑셀 파일 만들기

In [3]:
# wd = r"D:\3.자산\전산 dataset\230430\매각환매 채무상태 수정후"
# basedate = "230430"
# dir = wd  
dir = join(wd, basedate)

In [4]:
# 바로 작업시에는 이 셀 pass
# 원데이터 새로 읽기
files = ["2.솔림헬프_개인회생_원데이터_"+basedate, "2.솔림헬프_신용회복_원데이터_"+basedate, "2.솔림헬프_전체_원데이터_"+basedate]

# 1.개인회생
re_d = pd.read_excel(join(dir, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="차주").fillna("")
re_grt = pd.read_excel(join(dir, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="보증인").fillna("")

# 2.신용회복
credit_d = pd.read_excel(join(dir, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="차주").fillna("")
credit_grt = pd.read_excel(join(dir, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="보증인").fillna("")

# 날짜타입 칼럼
date_cols = credit_d.select_dtypes(include='datetime').columns
credit_d[date_cols] = credit_d[date_cols].astype(str)
date_cols = credit_grt.select_dtypes(include='datetime').columns
credit_grt[date_cols] = credit_grt[date_cols].astype(str)

# 3.무담보
raw_data = pd.read_excel(join(dir, files[2]+".xlsx"), dtype=account_dtype, sheet_name="리스트").fillna("")

In [ ]:
###################################################
# 무담보 채무상태 추가작업
# raw_data.loc[(raw_data.채무상태.str.contains(r"파산(?!\(기각\))|사망")) | (raw_data.담당자.str.contains(r"파산(?!\(기각\))|사망")) ,"새채무상태"] = "종료"
###################################################

In [5]:
# 풀데이터 읽기
pooldata = pd.read_excel(r"D:\3.자산\POOL별 관리자산\전체\0.전체정리_230602_채권구분&POOL표시.xlsx", sheet_name="POOL정리", dtype={"채무자키":str})

In [13]:
# POOL별 채무자키
pool_dict = {
            "POOL1" : pooldata.query('P1==True')["채무자키"],
            # "POOL2" : pooldata.query('P2==True')["채무자키"],
            "POOL3" : pooldata.query('P3==True')["채무자키"],
            "POOL4" : pooldata.query('P4==True')["채무자키"],
            "POOL5" : pooldata.query('P5==True')["채무자키"],
            "POOL6" : pooldata.query('P6==True')["채무자키"],
            # "POOL7" : pooldata.query('P7==True')["채무자키"],
            "POOL8" : pooldata.query('P8==True')["채무자키"],
            # "POOL9" : pooldata.query('P9==True')["채무자키"],
            "POOL10" : pooldata.query('P10==True')["채무자키"],
            "POOL11" : pooldata.query('P11==True')["채무자키"],
            "POOL12" : pooldata.query('P12==True')["채무자키"],
            "POOL13" : pooldata.query('P13==True')["채무자키"],
            "POOL14" : pooldata.query('P14==True')["채무자키"],
            "POOL15" : pooldata.query('P15==True')["채무자키"],
            "무차입" : pooldata.query('소속풀수==0')["채무자키"]
            }

##### 풀별 내부/발송 파일 만들기
- 일부 풀만 작업할 경우 위 pool_dict 주석처리를 통해

In [11]:
def summary_comment (ws1, file_kind, comp_closing, basedate) : 
    file_kind = file_kind
    start_row = None
    data = ["■ 자산확정일 : 20" + basedate[0:2]+"-"+basedate[2:4]+"-"+basedate[4:],
        "",
        "■ OPB 작성기준",
        "무담보 : 미상환원금잔액(현재원금+현재법비용)",
        "개인회생(인가) : 총분납잔금",
        "개인회생(미인가) : 미상환원금잔액",
        "신용회복(개인) : 상환후잔액(빈값인 경우 조정후합계)",
        "신용회복(프리) : 상환후잔액(빈값인 경우 조정후합계)",
        "신용회복(미확정) : 미상환원금잔액", # 8번 요소 
        "",
        "■ 제외(종료)기준 1 - 확실한 종료건",
        "  ▶ 종결일이 있거나 계좌메모에 종결여부가 표시되어있을 때",
        "  ▶ 채무상태나 담당자 값이 다음을 포함할 때 : " + comp_closing["종료"],
        "",
        "■ 제외(종료)기준 2 - 비정상건(매각 불가)",
        "  ▶ 채무상태나 담당자 값이 다음을 포함할 때 : " + comp_closing["시효"],
        "  ▶ 채무상태나 담당자 값이 다음을 포함하고 : " + comp_closing["면책"],
        "     정상 상태의 보증인이 한 명도 없을 때",
        "  ▶ 채무상태나 담당자 값이 다음을 포함하고 : " + comp_closing["불가"],
        "     정상 상태의 보증인이 한 명도 없을 때",
        "  ▶ 차주가 법인이고 정상 상태의 보증인이 한 명도 없을 때"
        ]
    if file_kind == "발송" :
        start_row = 13
        data = data[:9]
    else : 
        start_row = 15
    for i, row_data in enumerate(data) :
        ws1.cell(row=start_row + i, column=1,value=row_data)
    return ws1

def set_col_width_ws1(ws1) : 
    ws1.column_dimensions["A"].width = 18
    ws1.column_dimensions["B"].width = 10
    ws1.column_dimensions["C"].width = 14
    ws1.column_dimensions["D"].width = 10
    ws1.column_dimensions["E"].width = 14

In [14]:
pd.set_option('mode.chained_assignment', None)

output_dir = join(dir,"output")
if not os.path.exists(output_dir) : os.mkdir(output_dir)

for pool_kind, pool_keys in tqdm(pool_dict.items(), total=len(pool_dict))  : 
    
    ###### 내부용 만들기############################################################
    file_kind = "내부"
    # 풀 원데이터
    pool_raw = raw_data[raw_data.채무자키.isin(pool_keys)].copy()
    # 풀 무담보, 종료건
    pool_무담보 = pool_raw[pool_raw["새채무상태"].isin(["개인","법인"])]
    pool_비정상 = pool_raw[pool_raw["채권상태"]=="비정상"]
    pool_종료 = pool_raw[pool_raw["채권상태"]=="종료"]
    # 풀 조정건 : 새채무상태상 개회, 신복인 것만 고르기(기준데이터에 종결건 포함돼도 ok)
    re_d_keys = pool_raw[pool_raw["새채무상태"].str.contains("회생")].채무자키
    credit_d_keys = pool_raw[pool_raw["새채무상태"].str.contains("신용")].채무자키
    pool_re_d = re_d[re_d.채무자키.isin(pool_keys)]
    pool_re_grt = re_grt[re_grt.채무자키.isin(pool_keys)]
    pool_credit_d = credit_d[credit_d.채무자키.isin(pool_keys)]
    pool_credit_grt = credit_grt[credit_grt.채무자키.isin(pool_keys)]

    #순번 넣기
    pool_종료.insert(0, "순번", range(1,len(pool_종료)+1))
    pool_비정상.insert(0, "순번", range(1,len(pool_비정상)+1))
    pool_무담보.insert(0, "순번", range(1,len(pool_무담보)+1))
    pool_re_d.insert(0, "순번", range(1,len(pool_re_d)+1))
    pool_re_grt.insert(0, "순번", range(1,len(pool_re_grt)+1))
    pool_credit_d["No."] = range(1,len(pool_credit_d)+1)
    pool_credit_grt["No."] = range(1,len(pool_credit_grt)+1)

    ####### 요약시트[시작]
    summary = pool_raw.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex( \
        index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','보증인정상','시효','종료처리요망','종료'])
    summary.rename(columns={'계좌키':'건수', '반영OPB':"OPB"}, inplace=True)

    # 보증인 합산
    summary["건수(보증인)"] = [
        0,0, # 개인, 법인은 그대로
        pool_re_grt[pool_re_grt['인가/미인가']=="인가"].계좌키.count(), # 보증인은 없는 경우 있어서 그룹화하면 index에러날 수 있음
        pool_re_grt[pool_re_grt['인가/미인가']=="미인가"].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="개인")].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="프리")].계좌키.count(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].계좌키.count(), 
        0, 0, 0, 0 # 비정상3건과 종료
    ]
    summary["OPB(보증인)"] = [
        0,0, # 개인, 법인은 그대로
        pool_re_grt[pool_re_grt['인가/미인가']=="인가"].반영OPB.sum(),
        pool_re_grt[pool_re_grt['인가/미인가']=="미인가"].반영OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="개인")].반영OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="프리")].반영OPB.sum(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].반영OPB.sum(),
        0, 0, 0, 0 # 비정상3건과 종료
    ]

    # "종결" 행 제외한 열의 합계 계산
    row_sum = summary[summary.index != "종료"].sum()
    summary.loc["합계(종료제외)"] = row_sum
    summary = summary.reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','보증인정상','시효','종료처리요망',"합계(종료제외)",'종료'])

    summary = summary.fillna(0).astype(dtype='int64')
    summary.rename_axis("구분", inplace=True)
    

    ######### 엑셀 쓰기
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("비정상")
    ws8 = wb.create_sheet("종료")

    # 시트에 데이터 쓰기
    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    # a1에 구분 넣고 요약셀 2행 삭제
    ws1['a1'] = "구분"
    ws1.delete_rows(2)

    # 코멘트 작성하기
    ws1 = summary_comment(ws1, file_kind, comp_closing, basedate)

    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_비정상, index=False, header=True): ws7.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws8.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6') # 칼럼명
    fill_new = PatternFill(fill_type='solid', start_color='FF49C620', end_color='FF49C620') # 칼럼명에 '새'가 들어간 것
    # 테두리 : 직선
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8] :

        # 컬럼명 스타일 적용
        for cell in ws[1]:
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            if cell.value is not None and cell.value.startswith('새') : cell.fill = fill_new
            else : cell.fill = fill_col
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        # 요약시트
        max_row = 13 if ws.title == "요약" else ws.max_row
        # 다른시트
        for row in ws.iter_rows(min_row=2, max_row=max_row):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border

    # 요약시트 "계(종료제외)" 배경색
    for cell in ws1[12]:
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')

    # 요약시트 나머지행 글자크기
    for row in ws1.iter_rows(min_row=10) :
        for cell in row : 
            cell.font = Font(name='NN30', size=10)
    
    # 요약시트 컬럼 너비 지정하기(단위는 char임)
    set_col_width_ws1(ws1)

    # 파일 저장
    wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))


    ###### 발송용 만들기############################################################
    file_kind = "발송" ####

    pool_무담보.drop(['채권상태','새채무상태','채무상태','담당자','새보증인상태','회수합계','시효완성월','시효사유', '종결일', '메모'], axis=1, inplace=True)
    pool_무담보["성명"] = pool_무담보["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_무담보["주민등록번호"] = pool_무담보["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_re_d["분납자성명인"] = pool_re_d["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_re_grt["분납자성명인"] = pool_re_grt["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["고객명"] = pool_credit_d["고객명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["고객식별번호"] = pool_credit_d["고객식별번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_credit_grt["고객명"] = pool_credit_grt["고객명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_grt["고객식별번호"] = pool_credit_grt["고객식별번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_종료 = pd.concat([pool_비정상, pool_종료],)
    pool_종료["순번"] = range(1,len(pool_종료)+1)
    pool_종료 = pool_종료[["순번","채무자키","계좌키","채무상태","성명","주민등록번호"]]
    pool_종료["성명"] = pool_종료["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_종료["주민등록번호"] = pool_종료["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))

    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("종료")

    # 시트에 데이터 쓰기

    # 요약 시트 재작성
    summary.loc["종료"] += summary.loc["보증인정상"] + summary.loc["시효"] + summary.loc["종료처리요망"]
    summary.loc["합계(종료제외)"] -= summary.loc["보증인정상"] + summary.loc["시효"] + summary.loc["종료처리요망"]
    summary = summary.drop(["보증인정상", "시효", "종료처리요망"])
    summary.loc["종료","OPB"] = 0

    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    # a1에 구분 넣고 요약셀 2행 삭제
    ws1['a1'] = "구분"
    ws1.delete_rows(2)
    
    # 코멘트 작성하기
    ws1 = summary_comment(ws1, file_kind, comp_closing, basedate)

    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws7.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
    # 테두리
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7] :

        # 컬럼명 스타일 적용
        for cell in ws[1]: # 시트의 1행의 셀 반복
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            cell.fill = fill
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        # 요약시트
        max_row = 10 if ws.title == "요약" else ws.max_row
        # 다른시트
        for row in ws.iter_rows(min_row=2, max_row=max_row):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border
        
    # 요약시트 "합계(종료제외)" 배경색
    for cell in ws1[9]:
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')
    
    # 요약시트 나머지행 글자크기
    for row in ws1.iter_rows(min_row=10) :
        for cell in row : 
            cell.font = Font(name='NN30', size=10)

    # 요약시트 컬럼 너비 지정하기(단위는 char임)
    set_col_width_ws1(ws1)
            
    # 파일 저장
    wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))


100%|██████████| 13/13 [04:19<00:00, 19.95s/it]


In [ ]:
############## 끝 ###################

### 인적속성 - 다중채무, 다중담당 체크하기

In [301]:
####전산_check_전체계좌.ipynb 에도 있음
account = pd.read_pickle(r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847.pkl")
new_debtKey = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\새채무자키.pkl")[["새채무자키","채무자키","계좌키"]]
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

# 필요열만 새로운 df로 & 정렬
new_debtKey = account[["새채무자키", '채무자키','계좌키','채무상태','담당자']].copy()
new_debtKey.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

###### multi_index : 조작하기는 어렵고, 엑셀로 보낼때 자동으로 병합이 되어서 보기는 좋음
x = account.set_index(['새채무자키','채무자키','계좌키'],drop=True)["채무상태"]
# 다중채무, 다중담당 T/F값 입력
x = new_debtKey.groupby('새채무자키')
new_debtKey['다중채무상태여부'] = x['채무상태'].transform(lambda s: np.any(s != s.iloc[0]))
new_debtKey['다중담당여부'] = x['담당자'].transform(lambda s: np.any(s != s.iloc[0]))
# 다중인 것들만 
check = new_debtKey[new_debtKey["다중채무상태여부"] | new_debtKey['다중담당여부']].copy()
# index설정
gc = check.set_index(['새채무자키', '채무자키'], drop=True)
#저장
gc.to_excel(r"c://Users/SL/Desktop/check_채무상태(병합).xlsx")

### 시효(계좌)

In [14]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_account = "계좌조회새창_20230502_0847.pkl"
PATH_NEW_DEBTKEY = r"C:\Users\SL\Desktop\자산정리test파일\새채무자키.pkl"

account = pd.read_pickle(join(wd,filename_account))
new_debtKey = pd.read_pickle(PATH_NEW_DEBTKEY)
today = datetime.today().strftime("%y%m%d")

In [15]:
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

In [ ]:
# 필요열만 새로운 df로 & 정렬
account_limit_info = account[["새채무자키", '채무자키','계좌키','채무상태','담당자', "시효완성일", "시효완성월", "최초연체일", "최초시효완성일메모", "시효연장사유", "시효중단여부"]].copy()
account_limit_info.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

In [ ]:
# 동일한 새채무자키 시효중단여부 : 시효가 ##########################################작성중 중단
def set_extension_info(group):
    if 'Y' in group['시효중단여부'].values:
        group.loc[group['시효중단여부'] == '', '시효중단여부'] = 'Y'
        extension_reason = group.loc[group['시효중단여부'] == 'Y', '시효연장사유'].values[0]
        group['시효연장사유'] = extension_reason
    return group

account_limit_info = account_limit_info.groupby('새채무자키').apply(set_extension_info)


### 시효(법조치)

In [8]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_event = "법조치조회새창_20230522_2054.xlsx"
dtype_event = {"채무자키":str, "계좌키":str, "법조치키":str}
today = datetime.today().strftime("%y%m%d")

In [9]:
# 파일 읽기/필요칼럼추리기/원본파일pkl과 필요컬럼xlsx 파일 저장하기
df_event_ori = pd.read_excel(join(wd, filename_event), dtype=dtype_event)

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_event = pd.DataFrame(None, columns=["채무자키", "계좌키", "법조치키", "법조치구분", "법조치세부", "관할법원", "사건번호", "사건구분", "접수일", "확정일", "종료일",
    "법조치상태", "종국결과", "청구금액법원", "청구금액", "관련법조치관할법원", "관련법조치사건번호"])
    ###################################################### 여기까지 하고 계좌시트 작업으로 넘어감
df_event.채무자키 = df_event_ori.채무자키
df_event.계좌키 = df_event_ori.계좌키
df_event.신청일자 = df_event_ori.접수일
df_event.법무구분 = df_event_ori.법조치구분
df_event.관할법원 = df_event_ori.관할법원
df_event.사건번호 = df_event_ori.사건번호.str.replace(" ", "")
df_event.사건구분 = df_event_ori.사건구분
df_event.진행상태 = df_event_ori.종국결과
df_event.청구금액법원 = df_event_ori.청구금액법원
df_event.청구금액 = df_event_ori.청구금액
print(len(df_event.index))

####################
# 중복제거전 저장
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치_중복미제거.pkl"))
#df_event.to_excel(join(wd, "전처리완료", "전처리_법조치_중복미제거.xlsx"), index=False)
####################

# 채무자키로 전 계좌 불러오기
work_all_account = all_df[all_df.채무자키.isin(work_df.채무자키)].copy() # 객체 새로 만들지 않으면 계속 copywarning 나옴
# 전 계좌 중 작업 계좌에 있거나, 종결된 계좌면 Y, 아니면 N(솔림 잔존계좌)
cond = (work_all_account.계좌키.isin(work_df.계좌키)) | (work_all_account.종결일 != "")
work_all_account["계좌작업대상여부"] = np.where(cond, "Y", "N")

# 채무자키별로 전계좌 여부 확인 후 파일 저장하기
result = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = work_all_account.drop_duplicates('채무자키').copy()
result["채무자키"] = temp.채무자키.values
result["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result["담당자"] = temp.담당자.values
result["채무자명"] = temp.채무자명.values


# 작업대상여부의 유니크값이 1이면 모두 Y (모두 N인 경우는 논리적으로 불가능), 2면 작업계좌와 잔존계좌가 혼재한 것이므로 일부계좌
g = work_all_account.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result.iterrows() : 
     if g.get_group(v.채무자키).계좌작업대상여부.nunique() == 1 :
          result.loc[i, "전계좌여부"]="전계좌"
     else : result.loc[i, "전계좌여부"]="일부계좌"

# 저장하기
result.to_excel(path_save, index=False)